#check point 3 part 3

In [1]:
import numpy as np
import time

In [18]:
def load_from_text(file_name): 
  with open(file_name) as f:
    n = int(f.readline().strip('\n'))
    m = int(f.readline().strip('\n'))
    c_string = f.readline().replace('[', '').replace(']','').strip('\n')
    c = np.fromstring(c_string, dtype=int, sep=',')
    A = np.zeros((m,n),dtype=int)
    for i, string in enumerate(f.readline()[2:-3].split('],[')):
      for j in string.split(','):
        A[int(i),int(j)] = 1
  return A.T, c

In [19]:
def lab03_scp_duzhuolu(A,c,k=1):
  t = time.clock()
  j_len = len(A.T)
  s = np.zeros(j_len)
  b = A @ s
  while (b < k).any():
    u = np.full(j_len,np.inf)
    for j in range(j_len):
      if s[j] == 0:
        n = np.dot(A[:,j], b < k)
        if n > 0:
          u[j] = c[j] / n
    if (u < np.inf).any():
      s[u.argmin()] += 1
      b = b + A[:,u.argmin()]
  return s, np.dot(s,c), sum(s), (time.clock() - t)

In [35]:
def GreedySolveSetCover(file_name='/content/test.txt'):
  A, c = load_from_text(file_name)
  print(lab03_scp_duzhuolu(A,c))

In [36]:
GreedySolveSetCover()

(array([1., 1., 1., 1., 0.]), 12.0, 4.0, 0.0006360000000000809)


# Check Point 4 BDD generation(before reduction) algorithm

In [2]:
def BDDSolveSetCover(A,c):
  n = len(A)
  m = len(A.T)
  Nodes = np.array([],dtype=np.int16)
  Arcs = np.array([],dtype=np.int16)
  node_count = 0
  Nodes = np.append(Nodes,{0: np.zeros(n,dtype=np.int16)})
  for j in range(m-1):
    Nodes_newLayer = {}
    for node, state in Nodes[j].items():
      for k in [0,1]:
        if k:
          new_state = (state + A[:,j]) > 0
        else:
          new_state = state
        if not ((new_state+A[:,j+1:].sum(axis=1)) == 0).any():
          reach_end = True
          for node1, state1 in Nodes_newLayer.items():
            if (state1 == new_state).all():
              Arcs = np.append(Arcs, np.array([node, node1, k*c[j]]))
              reach_end = false
              break
          if reach_end:
            node_count+=1
            Nodes_newLayer[node_count] = new_state
            Arcs = np.append(Arcs, np.array([node, node_count, k*c[j]]))
    Nodes = np.append(Nodes, Nodes_newLayer)
  node_count += 1
  Nodes = np.append(Nodes, {node_count: np.ones(n)})

  for node, state in Nodes[m-1].items():
    for k in [0,1]:
      if k:
        new_state = state + A[:,-1]
      else:
        new_state = state
      if not (new_state == 0).any():
        Arcs = np.append(Arcs, np.array([node, node_count, k*c[-1]]))

  return np.array(range(0,6)), Arcs.reshape(-1,3)

In [3]:
A = np.array([
              [1,1,1],
              [1,0,1],
              [0,1,1],
              [1,1,0]
])

c = np.array([5,3,3])

In [4]:
nodes, arcs = BDDSolveSetCover(A,c)

In [5]:
nodes

array([0, 1, 2, 3, 4, 5])

In [6]:
arcs

array([[0, 1, 0],
       [0, 2, 5],
       [1, 3, 3],
       [2, 4, 0],
       [2, 5, 3],
       [3, 6, 3],
       [4, 6, 3],
       [5, 6, 0],
       [5, 6, 3]])

First collum: start node of the arc <br>Second collum: end node of the arc<br>Third collum: distance of the arc